In [1]:
import keras as k
from keras.layers import merge
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from keras.callbacks import History
from keras.layers import Activation
from keras.models import model_from_json
from keras.optimizers import Adam
from matplotlib import pyplot as plt
from scipy.ndimage import rotate as rot
from sklearn.model_selection import train_test_split
from keras import utils
import numpy as np
import tensorflow as tf


def my_autoencode(img_shape, code_size=32):
    H,W,C = img_shape
    
    # encoder
    encoder = k.models.Sequential()
    encoder.add(k.layers.InputLayer(img_shape))
    encoder.add(k.layers.Conv2D(32, (3, 3), activation='elu', padding='same'))
    encoder.add(k.layers.MaxPooling2D((2, 2), padding='same'))
    encoder.add(k.layers.Conv2D(64, (3, 3), activation='elu', padding='same'))
    encoder.add(k.layers.MaxPooling2D((2, 2), padding='same'))
    encoder.add(k.layers.Conv2D(64, (3, 3), activation='elu', padding='same'))
    encoder.add(k.layers.MaxPooling2D((2, 2), padding='same'))
    encoder.add(k.layers.Conv2D(128, (3, 3), activation='elu', padding='same'))
    encoder.add(k.layers.AveragePooling2D((2, 2), padding='same'))
    encoder.add(k.layers.Flatten())
    encoder.add(k.layers.Dense(512, activation='elu'))
    encoder.add(k.layers.Dense(256, activation='elu'))
    encoder.add(k.layers.Dense(code_size, activation='elu'))
    encoder.summary()

    # decoder
    decoder = k.models.Sequential()
    decoder.add(k.layers.InputLayer((code_size,)))
    decoder.add(k.layers.Dense(256, activation='elu'))
    decoder.add(k.layers.Dense(512, activation='elu'))
    decoder.add(k.layers.Dense(8192, activation='elu'))
    decoder.add(k.layers.Reshape((8, 8, 128)))
    decoder.add(k.layers.UpSampling2D((2, 2)))
    decoder.add(k.layers.Conv2DTranspose(128, kernel_size=(3, 3), activation='elu', padding='same'))
    decoder.add(k.layers.UpSampling2D((2, 2)))
    decoder.add(k.layers.Conv2DTranspose(64, kernel_size=(3, 3), activation='elu', padding='same'))
    decoder.add(k.layers.UpSampling2D((2, 2)))
    decoder.add(k.layers.Conv2DTranspose(64, kernel_size=(3, 3), activation='elu', padding='same'))
    decoder.add(k.layers.UpSampling2D((2, 2)))
    decoder.add(k.layers.Conv2DTranspose(32, kernel_size=(3, 3), activation='elu', padding='same'))
    decoder.add(k.layers.Conv2DTranspose(3, kernel_size=(3, 3), activation='elu', padding='same')) # Unsure about this
    decoder.summary()
    
    return encoder, decoder

all_data = np.load('images_dataset.npy')
X_train = all_data[:2723, :]
X_test = all_data[2723:, ]
shape = X_train[0].shape # Get from dataset
encoder, decoder = my_autoencode(shape, code_size=128)
inp = k.layers.Input(shape)
code = encoder(inp)
reconstruction = decoder(code)
autoencoder = k.models.Model(inputs=inp, outputs=reconstruction)
autoencoder.compile(optimizer="adamax", loss='mse')
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('hackathon_autoencoder.{epoch:02d}-{val_loss:.2f}.h5', verbose=1, save_best_only=True)
]


# If you want to resume from a checkpoint
#     import keras.backend as K
#     def reset_tf_session():
#         K.clear_session()
#         tf.reset_default_graph()
#         s = K.get_session()
#         return s
#     #### uncomment below to continue training from model checkpoint
#     #### every time epoch counter starts at 0, so you need to track epochs manually
#     from keras.models import load_model
#     s = reset_tf_session()
#     autoencoder = load_model("checkpoints/hackathon_autoencoder.78-508.84.h5")  # continue after epoch 0+1
#     encoder = autoencoder.layers[1]
#     decoder = autoencoder.layers[2]

# # Train Model


C:\Users\sitandon\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 128)       73856     
__________

In [2]:
autoencoder.fit(x=X_train, y=X_train,
                validation_data=[X_test, X_test],
                epochs=200,
                batch_size=32,
                shuffle=True,
                callbacks = callbacks
               )

Train on 2723 samples, validate on 303 samples
Epoch 1/200
2723/2723 [==============================] - 40s 15ms/step - loss: 7217.8175 - val_loss: 3020.7841

Epoch 00001: val_loss improved from inf to 3020.78407, saving model to hackathon_autoencoder.01-3020.78.h5
Epoch 2/200
2723/2723 [==============================] - 28s 10ms/step - loss: 2188.1803 - val_loss: 2433.5588

Epoch 00002: val_loss improved from 3020.78407 to 2433.55877, saving model to hackathon_autoencoder.02-2433.56.h5
Epoch 3/200
2723/2723 [==============================] - 28s 10ms/step - loss: 1618.7508 - val_loss: 1418.9019

Epoch 00003: val_loss improved from 2433.55877 to 1418.90192, saving model to hackathon_autoencoder.03-1418.90.h5
Epoch 4/200
2723/2723 [==============================] - 28s 10ms/step - loss: 1335.6574 - val_loss: 1262.6507

Epoch 00004: val_loss improved from 1418.90192 to 1262.65073, saving model to hackathon_autoencoder.04-1262.65.h5
Epoch 5/200
2723/2723 [==============================] -

2723/2723 [==============================] - 32s 12ms/step - loss: 429.9614 - val_loss: 559.9329

Epoch 00042: val_loss improved from 569.67974 to 559.93290, saving model to hackathon_autoencoder.42-559.93.h5
Epoch 43/200
2723/2723 [==============================] - 32s 12ms/step - loss: 402.0951 - val_loss: 560.9839

Epoch 00043: val_loss did not improve from 559.93290
Epoch 44/200
2723/2723 [==============================] - 32s 12ms/step - loss: 396.3365 - val_loss: 535.8517

Epoch 00044: val_loss improved from 559.93290 to 535.85172, saving model to hackathon_autoencoder.44-535.85.h5
Epoch 45/200
2723/2723 [==============================] - 32s 12ms/step - loss: 392.4922 - val_loss: 584.5097

Epoch 00045: val_loss did not improve from 535.85172
Epoch 46/200
2723/2723 [==============================] - 32s 12ms/step - loss: 404.7217 - val_loss: 550.5380

Epoch 00046: val_loss did not improve from 535.85172
Epoch 47/200
2723/2723 [==============================] - 33s 12ms/step - los

2723/2723 [==============================] - 44s 16ms/step - loss: 247.6757 - val_loss: 527.6314

Epoch 00090: val_loss did not improve from 519.31886
Epoch 91/200
2723/2723 [==============================] - 44s 16ms/step - loss: 242.3918 - val_loss: 552.0657

Epoch 00091: val_loss did not improve from 519.31886
Epoch 92/200
2723/2723 [==============================] - 44s 16ms/step - loss: 232.5682 - val_loss: 544.5623

Epoch 00092: val_loss did not improve from 519.31886
Epoch 93/200
2723/2723 [==============================] - 44s 16ms/step - loss: 234.7119 - val_loss: 550.3197

Epoch 00093: val_loss did not improve from 519.31886
Epoch 94/200
2723/2723 [==============================] - 46s 17ms/step - loss: 239.2253 - val_loss: 536.5065

Epoch 00094: val_loss did not improve from 519.31886
Epoch 95/200
2723/2723 [==============================] - 47s 17ms/step - loss: 233.3875 - val_loss: 549.7968

Epoch 00095: val_loss did not improve from 519.31886
Epoch 96/200
2723/2723 [=====

2723/2723 [==============================] - 65s 24ms/step - loss: 174.5407 - val_loss: 569.9338

Epoch 00140: val_loss did not improve from 519.31886
Epoch 141/200
2723/2723 [==============================] - 65s 24ms/step - loss: 172.4167 - val_loss: 556.3219

Epoch 00141: val_loss did not improve from 519.31886
Epoch 142/200
2723/2723 [==============================] - 64s 24ms/step - loss: 169.3352 - val_loss: 573.9597

Epoch 00142: val_loss did not improve from 519.31886
Epoch 143/200
2723/2723 [==============================] - 65s 24ms/step - loss: 172.8431 - val_loss: 573.7021

Epoch 00143: val_loss did not improve from 519.31886
Epoch 144/200
2723/2723 [==============================] - 64s 24ms/step - loss: 174.3868 - val_loss: 568.8549

Epoch 00144: val_loss did not improve from 519.31886
Epoch 145/200
2723/2723 [==============================] - 64s 24ms/step - loss: 168.0164 - val_loss: 564.0805

Epoch 00145: val_loss did not improve from 519.31886
Epoch 146/200
2723/2723 

2723/2723 [==============================] - 64s 24ms/step - loss: 136.4714 - val_loss: 583.9222

Epoch 00190: val_loss did not improve from 519.31886
Epoch 191/200
2723/2723 [==============================] - 64s 24ms/step - loss: 136.9461 - val_loss: 611.1627

Epoch 00191: val_loss did not improve from 519.31886
Epoch 192/200
2723/2723 [==============================] - 65s 24ms/step - loss: 139.5341 - val_loss: 582.4551

Epoch 00192: val_loss did not improve from 519.31886
Epoch 193/200
2723/2723 [==============================] - 64s 24ms/step - loss: 140.3546 - val_loss: 589.0793

Epoch 00193: val_loss did not improve from 519.31886
Epoch 194/200
2723/2723 [==============================] - 64s 24ms/step - loss: 137.2948 - val_loss: 583.8648

Epoch 00194: val_loss did not improve from 519.31886
Epoch 195/200
2723/2723 [==============================] - 64s 24ms/step - loss: 139.5748 - val_loss: 603.6452

Epoch 00195: val_loss did not improve from 519.31886
Epoch 196/200
2723/2723 